### -- I ) Projet analyse des données immobilière, récupération et traitement des données --

L'objectif du projet est de récupérer des données sur le prix de l'immobilier 
pour afficher l'évolution de son prix par jour et par mois.

Import des librairies

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd 
from datetime import datetime
import os

/Users/gj3dgf4hf/Documents/Projet_Python/Figaro-immo/env_Python_3.9/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Vide le dossier appelé "page_html" avant son alimentation.

In [2]:
taile_du_dossier = os.listdir("page_Html")

if(len(taile_du_dossier) > 0):
    for page in range(1,100+1):
        os.remove(f"page_Html/immo{page}.html")
else:
    pass

Requete et récupération des la pages en HTML

In [3]:

#-------------------------------- boucle pour récuperer les pages numérotées.
for page in range(1,100+1):
    # Requete de la page
    url = f'https://immobilier.lefigaro.fr/annonces/immobilier-vente-bien-france.html?page={page}'
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, 'html.parser')
    
    # Enregistrement de la page
    file_html = open(f"page_Html/immo{page}.html","w")
    file_html.write(str(soup))

Ouveture du fichier / récupération des données ( prix , type de logement, ville)

In [ ]:
l_prix_logement = []
l_ville = []
l_type_de_bien = []

for i in range(1,100+1):
    with open(f"page_Html/immo{i}.html") as file:
        soup2 = BeautifulSoup(file, "html.parser")
    
    print(f"page_Html/immo{i}.html")

    prix_logement = soup2.find_all("span", class_="price")
    ville = soup2.find_all("span", class_="title-location")
    type_de_bien = soup2.find_all("h2", class_="cartouche-liste__title")

    # Add element in list 
    
    for index in range(0, len(prix_logement) ):

        # Nettoyage du prix
        u_prix = prix_logement[index].text
        u_prix = u_prix.replace("€", "")
        u_prix = u_prix.replace(" ", "")
        l_prix_logement.append(u_prix)
       
        # Nettoyage du nom de la ville
        u_ville = ville[index].text
        u_ville = u_ville[:-4]
        l_ville.append(u_ville)

        # Nettoyage du type de bien
        u_type_de_bien = type_de_bien[index].text
        categorie_bien = re.search('[a-zA-Z0-9_]+', u_type_de_bien )
        l_type_de_bien.append(categorie_bien.group())



Création du dataframe contenant les données [ Ville, Type de bien, Prix ].

In [5]:
dataframe = pd.DataFrame({"Ville": l_ville,
                "Type de bien":l_type_de_bien,
                "Prix":l_prix_logement})

Visualisation du dataframe

In [6]:
dataframe.head(5)

,Ville,Type de bien,Prix
0,Toulon,maison,960000
1,Saint-Ouen,appartement,395000
2,Paris 10ème,appartement,353600
3,Neuilly-sur-Seine,appartement,380000
4,Champigny-sur-Marne,appartement,290000


Enregistrement du dataframe au format CSV

In [7]:
dataframe.to_csv("rendu_csv/immo_figaro.csv", index=False)